# <span style="color:darkblue"> Lecture 16 - Practicing Chaining </span>

<font size = "5">

In this lecture you will get a chance to practice <br>
the main dataset operations

- There will be a quiz on this lecture

# <span style="color:darkblue"> I. Import Libraries and Data </span>


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
results  = pd.read_csv("data_raw/results.csv")
races    = pd.read_csv("data_raw/races.csv")
results["points col"] = results["points"]

# <span style="color:darkblue"> II. Review Dataset Operations </span>

<font size = "5">

See attached file for a refresher on syntax

```[] ``` $\qquad \qquad \qquad \quad$: Extracting columns <br>
```.query() ``` $\qquad \qquad $: Subsetting rows <br>
```.recode() ``` $ \qquad \quad \ \ $: Replacing values <br>
```.groupby().agg() ```: Aggregate statistics by subgroup <br>
```.rename() ``` $\qquad \quad \ \ $: Change name of columns

Full list:

<font size = "4">

https://www.w3schools.com/python/pandas/pandas_ref_dataframe.asp

# <span style="color:darkblue"> III. Examples of Chaining </span>

<font size = "5">

The operations with "." are read left to right

- Combine any of the above operations
- Great way to make code efficient
- The sky's the limit!


Subsetting **before** extracting columns

In [ ]:
# Get data for drivers that scored more than 20 points on individual races
# Then extract the columns "driverId" and "points"
results.query('points >= 20')[["driverId","points"]]

<font size = "5">

Subsetting **before** aggregating

In [ ]:
# This obtains a subset of drivers who competed in races 500 onwards
# then computes the average by team ("constructorId")

(results.query('raceId >= 500')
        .groupby("constructorId")
        .agg(mean_points = ("points","mean")))


<font size = "5">

Subsetting **after** aggregating

In [ ]:
# This obtains the average points by team ("constructorId"), then 
# produces a subset of team whose average is higher than 10

(results.groupby("constructorId")
        .agg(mean_points = ("points","mean"))
        .query('mean_points >= 10'))

<font size = "5">

Chaining inside queries + NaNs

In [ ]:
# "is.na()" produces a True/False vector, checking for missing values
# "is.notna()" produces a True/False vector, checking for non-missing values
# .str.isnumeric()
results["points"].isna()
results["points"].notna()

subset_nas    = results.query('points.isna()')
subset_nonnas = results.query('points.notna()')


## <span style="color:darkblue"> III. Quiz Structure </span>

<font size = "5">

The day of the quiz I will ...
- Provide a dataset with information
- Give more specific instructions.
- Below, you will see the type of questions that will be asked.
- The idea is for you to apply known concepts to new data
- You have 50 minutes to complete the assignment

Questions

(exact wording may change in quiz, but exercise will be very similar)



<font size = "5">

(a) Replace the values of a column

- Obtain unique string values of a column
- Use the ".replace()" command

Hint: See Lecture 13

In [ ]:
results = pd.read_csv("data_raw/results.csv")

subset      = results.query("milliseconds.str.isnumeric() == False")
list_unique = pd.unique(subset["milliseconds"])
print(list_unique)
list_old = ['\\N']
list_new = [np.nan]
results["milliseconds"] = results["milliseconds"].replace(list_old, list_new) #replaces vals
results["milliseconds_numeric"] = pd.to_numeric(results["milliseconds"]) # converts col to numeric

<font size = "5">

(b) Recode a numeric column

- Use the "pd.cut()" command to create <br>
a new column based on an interval.
- See Lecture 13 for more details

In [ ]:
circuits = pd.read_csv("data_raw/circuits.csv")

bins_x = [0,2500, 5000]
labels_x = ["Between 0 and 2500",
            "Between 2500 and 5000"]

#tells u what's old and new
subset      = circuits.query("alt.str.isnumeric() == False")
list_unique = pd.unique(subset["alt"])
print(list_unique)

#replacing
list_oldy = ['\\N','-7']
list_newy = [np.nan,-7]
circuits["alt"] = circuits["alt"].replace(list_oldy, list_newy)
circuits["alt_numeric"] = pd.to_numeric(circuits["alt"])

#change to numeric


results["bins_alt"] = pd.cut(circuits["alt_numeric"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)

#       float("inf") and float("-inf") represent infinity and negative infinity
#       The "right" command indicates that the right interval is
#       "less than or equal to"

<font size = "5">

(c) Aggregate and query

- Use a combniation of the following commands <br>
to produce a new dataset <br>
``` .query() ``` <br>
``` .groupby().agg() ``` <br>

In [ ]:
results_agg_2 = (results.query("raceId >= 200")
                       .groupby("constructorId")
                        .agg(mean_laps = ('laps','mean'),
                           sd_laps =   ('laps','std')))

results_agg_2

<font size = "5">

(d) Aggregate and sort

- Use a combniation of the following commands <br>
to produce a new dataset <br>
``` .groupby().agg() ``` <br>
``` .sort_values() ```

In [ ]:
results_agg = (results.groupby("constructorId")
                      .agg(mean_laps = ('laps','mean'),
                           sd_laps =   ('laps','std'))
                           .sort_values (by = "mean_laps", ascending = False))

results_agg

<font size = "5">

(e) Rename column

- Create a dictionary
- Rename one or more columns in a dataset <br>
using the dictionary

In [ ]:

car_dictionary = {"car_model": ["Ferrari","Tesla","BMW"],
                  "year":      ["2018","2023","2022"] }


races_raw    = pd.read_csv("data_raw/races.csv")

#renaming one col
dict_rename_circuits = {"name": "circuit_name"}
circuits = circuits.rename(columns = dict_rename_circuits)
print (races.columns.values)

#renaming multiple
#dict_rename_pitfall = {"name_x": "race_name",
#                       "name_y" : "circuit_name"} #sep with comma
#races_merge_pitfalls = races_raw.rename(columns = dict_rename_pitfall)

#print (races_merge_pitfalls.columns.values)


<font size = "5">

(f) Merge dataset

- Use "pd.merge" to combine two datasets: <br>
a primary and secondary
- Only merge a subset of the columns of the <br>
secondary dataset
- Use "display" to show a the merged dataset,  <br>
extracting a subset of the columns

In [ ]:

races_merge = pd.merge(races_raw,
                       circuits[["circuitId","circuit_name"]], # multiple
                       on = "circuitId",
                       how = "left")

display (races_merge[["raceId", "circuitId", "circuit_name"]].sort_values(by = "circuitId"))

